In [1]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler 
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


Using TensorFlow backend.


In [2]:
#Basic preprocessing of data
train_data=pd.read_excel('Train_dataset.xlsx')
test_data=pd.read_excel('Test_dataset.xlsx')

train_x=train_data.iloc[:,:-1].values

train_y=train_data.iloc[:,-1].values

x_test=test_data.iloc[:,:].values
original_test=x_test


train_x=np.delete(train_x,[7,8,9,665],0)
train_x=np.delete(train_x,[0,1,2,4],1)
train_y=np.delete(train_y,[7,8,9,665],0)

x_test=np.delete(x_test,[0,1,2,4],1)

df=pd.DataFrame(train_x)

In [3]:
#handling the missing values of SWM column in test set
for i in range(len(x_test[:,4])):
    
    if(type(x_test[:,4][i])==str):
        continue
    else:
        x_test[:,4][i]="LOW"

In [4]:
from sklearn.preprocessing import LabelEncoder

labelencoder_X = LabelEncoder()
train_x[:,4] = labelencoder_X.fit_transform(train_x[:, 4])
x_test[:,4]=labelencoder_X.fit_transform(x_test[:,4])


In [5]:
#Filling the nan entries of columns(with less missing values) of training set with median of values
impute=SimpleImputer(missing_values=np.nan,strategy='median')

impute=impute.fit(train_x[:,0:7])
train_x[:,0:7]=impute.transform(train_x[:,0:7])

impute=impute.fit(train_x[:,(9,10)])
train_x[:,(9,10)]=impute.transform(train_x[:,(9,10)])


In [6]:
#Filling the nan entries of columns of training set with median of values

impute_test=SimpleImputer(missing_values=np.nan,strategy='median')
impute=impute.fit(x_test)
x_test=impute.transform(x_test)


In [7]:
#Now using linear regression for filling the reamaining missing values in the dataset
y=train_x[:,8]
x=train_x[:,0]

y=list([y[i] for i in range(len(y)) if(~np.isnan(y[i]))])
x=list([x[i] for i in range(len(y)) if(~np.isnan(y[i]))])

l=len(y)
l1=len(x)

x=np.array(x)
y=np.array(y)

x=np.reshape(x,(l1,1))
y=np.reshape(y,(l,1))

y1=train_x[:,7]
x1=train_x[:,3]

y1=list([y1[i] for i in range(len(y1)) if(~np.isnan(y1[i]))])
x1=list([x1[i] for i in range(len(y1)) if(~np.isnan(y1[i]))])

l=len(y1)
l1=len(x1)

x1=np.array(x1)
y1=np.array(y1)

x1=np.reshape(x1,(l1,1))
y1=np.reshape(y1,(l,1))
final=np.zeros((783,))

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x, y)

for i in range(783):
    if(np.isnan(train_x[:,8][i])):
        z=train_x[:,0][i]
        final[i]=regressor.predict([[z]])
    else:
        final[i]=train_x[:,8][i]

train_x[:,8]=final


regressor1 = LinearRegression()
regressor1.fit(x1, y1)

for i in range(783):
    if(np.isnan(train_x[:,7][i])):
        z=train_x[:,3][i]
        final[i]=regressor.predict([[z]])
    else:
        final[i]=train_x[:,7][i]

train_x[:,7]=final

In [8]:
#Scaling the data using Standard Scalar
sc_x = StandardScaler()

train_x=sc_x.fit_transform(train_x)
x_test=sc_x.transform(x_test)


In [9]:
model=Sequential()
model.add(Dense(128,input_dim=11,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='linear'))

optimizer=keras.optimizers.RMSprop(0.001)
model.compile(loss='mse',optimizer=optimizer,metrics=['mae','mse'])

In [10]:
model.fit(train_x,train_y,validation_split=0.2,epochs=1000,verbose=0)

In [11]:
y_pred=model.predict(x_test)


In [12]:
import csv

with open('hackathon.csv', mode='w') as hackathon_file:
    hackathon_writer = csv.writer(hackathon_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    hackathon_writer.writerow(["City","Covid cases"])
    for i in range(len(x_test)):
      hackathon_writer.writerow([original_test[i,0],y_pred[i]])

        